In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
import io

## ICU Projections

In [ ]:
url = "https://docs.google.com/spreadsheets/d/11y3g_qr05H6u6gDorZKUsJKWLoq6KerOfhPYOsnI55Q/export?format=csv&id=11y3g_qr05H6u6gDorZKUsJKWLoq6KerOfhPYOsnI55Q&gid=0"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=2048341719"
s=requests.get(url).content
dft = pd.read_csv(io.StringIO(s.decode('utf-8')))
dft.head()

In [ ]:
dft.case_status.replace("suspected_covid","Suspected Covid",inplace=True)
dft.case_status.replace("confirmed_positive","Confirmed Covid",inplace=True)

config = {'displayModeBar': False}

fig = go.Figure()

types = dft.case_status.unique()

for thing in types:
    temp = dft.loc[dft.case_status==thing]
    fig.add_trace(go.Bar(name=thing,x=temp.date, y=temp.number))

    
fig.add_trace(go.Scatter(name='Worst Case',x=df.Date,y=df['Worst Case'],opacity=.3,marker_color='red'))
fig.add_trace(go.Scatter(name='Best Case',x=df.Date,y=df['Best Case'],opacity=.3,marker_color='blue'))

fig.update_layout(barmode='stack')

fig.update_layout(
    yaxis =  {'gridcolor': '#d1cfc8', 'title': 'Confirmed Covid Cases In ICU'},
    xaxis = {'showgrid': False},      
    title={'text':f"COVID ICU Cases",
            'y':1,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor="White",
    legend_orientation="h"
)


## Total Cases

In [17]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1430697335"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Deaths,New deaths,Under Investigation,Positives,New positives,Negatives,Total tested,New tests,Resolved,Positive pct,Negative pct,Investigation pct,New Positive pct,Hospitalized,ICU,Ventilator
0,2020-03-02,0,0,12,18,0,975,1005,0,3,0.017910,0.970149,0.011940,NaN,NaN,NaN,NaN
1,2020-03-03,0,0,45,20,2,1061,1126,121,3,0.017762,0.942274,0.039964,0.016529,NaN,NaN,NaN
2,2020-03-04,0,0,102,20,0,1216,1338,212,3,0.014948,0.908819,0.076233,0.000000,NaN,NaN,NaN
3,2020-03-05,0,0,78,22,2,1466,1566,228,4,0.014049,0.936143,0.049808,0.008772,NaN,NaN,NaN
4,2020-03-06,0,0,72,26,4,1665,1763,197,4,0.014748,0.944413,0.040839,0.020305,NaN,NaN,NaN


In [23]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['New tests'].tail(1).values[0],
    number = {'font': {'size': 60}},))

fig.add_trace(go.Scatter(x=df.Date,y=df['New tests'],line=dict(color='red', width=3),visible=True,opacity=0.5,name="Value"))
fig.add_trace(go.Scatter(x=df.Date,y=df['New tests'].rolling(7).mean(),line=dict(color='#FFF', dash='dot'), opacity=0.5,name="7 Day Average"))


fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"<span style='font-size:0.8em>New Tests</span><br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['New tests'].iloc[-2]}},
]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True, 'tickformat':'%d-%B'},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#FFF"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#343332',
    paper_bgcolor="#343332",
    legend_orientation="h",
    )



In [ ]:
fig = go.Figure()
temp = df.loc[df['Total tested'].notna()]


fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Total tested'].tail(1).values[0],

    )
             )

fig.add_trace(go.Scatter(x=temp.Date,y=temp['Total tested'],marker_color='#5E5AA1', visible=False))

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Total Tested<br><span style='font-size:0.5em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta",
        'delta' : {'reference': df['Total tested'].iloc[-2],
                  'relative': True, 'position' : "top"}}]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",

    updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Total tested Over Time"},]),
        ]),
    )
])


In [ ]:
fig = go.Figure()
temp = df.loc[df['New Positive pct'].notna()]
temp = df.loc[df['New Positive pct'] > 0]

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['New Positive pct'].tail(1).values[0]*100,
    number = {'font': {'size': 60}},
    ))

fig.add_trace(go.Scatter(x=temp.Date,y=temp['New Positive pct'],line=dict(color='#000', dash='dot'),visible=True, opacity=0.5, name="Value"))
fig.add_trace(go.Scatter(x=df.Date,y=temp['New Positive pct'].rolling(7).mean(),line=dict(color='#5E5AA1',width=5), opacity=0.5,name="7 Day Average"))



fig.update_layout(
    template = {'data' : {'indicator': [{
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['New Positive pct'].iloc[-2]*100,
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}},
        ]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True, 'tickformat':'%d-%B'},
    yaxis = {'showgrid': False,'visible':True},
    title={'text': f"New Positive %<br><span style='font-size:0.5em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>",
            'y':0.95,
            'x':0.15,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",

        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",
)

In [ ]:
fig = go.Figure()
temp = df.loc[df['Total tested'].notna()]

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Under Investigation'].tail(1).values[0],))


fig.add_trace(go.Scatter(x=temp.Date,y=temp['Under Investigation'],marker_color='#5E5AA1', visible=False))

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Under Investigation<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Under Investigation'].iloc[-2],
                  'increasing': {'color':'grey'},
                  'decreasing': {'color':'grey'}}}, 
        ]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",

    updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Under Investigation Over Time"},]),
        ]),
    )
])


In [ ]:
fig = go.Figure()
temp = df.loc[df['Ventilator'].notna()]
fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = temp['Ventilator'].tail(1).values[0],))

fig.add_trace(go.Scatter(x=temp.Date,y=temp['Ventilator'],marker_color='#54CAF1',visible=False))

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"On Ventilator<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Ventilator'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False},      
    title={'text':"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E4F7FD',
    paper_bgcolor="#E4F7FD",
    legend_orientation="h",
    updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "On Ventilator Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()
temp = df.loc[df['ICU'].notna()]
fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = temp['ICU'].tail(1).values[0],))
fig.add_trace(go.Scatter(x=temp.Date,y=temp['ICU'],marker_color='#54CAF1',visible=False))

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"In ICU<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['ICU'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False},      
    title={'text':"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E4F7FD',
    paper_bgcolor="#E4F7FD",
    legend_orientation="h",
    updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "In ICU Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()
temp = df.loc[df['Hospitalized'].notna()]
fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = temp['Hospitalized'].tail(1).values[0],))
fig.add_trace(go.Scatter(x=temp.Date,y=temp['Hospitalized'],marker_color='#54CAF1',visible=False))

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"In Hospital<br><span style='font-size:0.5em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Hospitalized'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False},      
    title={'text':"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E4F7FD',
    paper_bgcolor="#E4F7FD",
    legend_orientation="h",
    updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Hospitalized Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Positives'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Total Cases<br><span style='font-size:0.5em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Positives'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['Positives'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Cases Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['New positives'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"New Cases<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['New positives'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['New positives'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "New Cases Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Deaths'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Total Deaths<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Deaths'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['Deaths'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Deaths Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['New deaths'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"New Deaths<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['New deaths'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['New deaths'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "New deaths Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Resolved'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Recovered Cases<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Resolved'].iloc[-2],
                  }}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['Resolved'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Recovered Cases Over Time"},]),
        ]),
    )])

In [ ]:
# LHIN Population source : https://www.arcgis.com/home/item.html?id=7d11e1ba908840f79e827947a564e7da
LHIN= {
    "Erie St. Clair": 643432,
    "South West":958298,
    "Waterloo Wellington":758144,
    "Hamilton Niagara Haldimand Brant":1414803,
    "Central West":840086,
    "Mississauga Halton":1182471,
    "Toronto Central":1147368,
    "Central":1798751,
    "Central East":1607766,
    "South East":491676,
    "Champlain":1260528,
    "North Simcoe Muskoka":462026,
    "North East":566492,
    "North West":283760,  
}

In [ ]:
PHU = {
    "Erie St. Clair": ['Chatham-Kent Health Unit', 'Lambton Health Unit', 'Windsor-Essex County Health Unit'],
    "South West":['Middlesex-London Health Unit', 'Grey Bruce Health Unit', 'Haldimand-Norfolk','Southwestern','Huron County Health Unit'],
    "Waterloo Wellington":['Wellington-Dufferin-Guelph Health Unit','Grey Bruce Health Unit','Waterloo Health Unit'],
    "Hamilton Niagara Haldimand Brant":['Brant County Health Unit','City of Hamilton Health Unit','Halton Regional Health Unit','Haldimand-Norfolk','Niagara Regional Area Health Unit'],
    "Central West":['Wellington-Dufferin-Guelph Health Unit','Toronto Public Health'],
    "Mississauga Halton":['Peel Regional Health Unit','Halton Regional Health Unit','Toronto Public Health'],
    "Toronto Central":['City of Toronto Health Unit'],
    "Central":['York Regional Health Unit','Toronto Public Health'],
    "Central East":['Peterborough County-City Health Unit','Haliburton, Kawartha','Pine Ridge District Health Unit','Toronto Public Health','Durham Regional Health Unit'],
    "South East":['Hastings and Prince Edward Counties Health Unit','Leeds, Grenville and Lanark District Health Unit','Kingston, Frontenac, and Lennox and Addington Health Unit','Haliburton, Kawartha, Pine Ridge District Health Unit'],
    "Champlain":['Leeds Grenville and Lanark','The Eastern Ontario Health Unit','City of Ottawa Health Unit','Renfrew'],
    "North Simcoe Muskoka":['Simcoe Muskoka District Health Unit','Grey Bruce Health Unit'],
    "North East":['Northwestern Health Unit','Timiskaming Health Unit','North Bay Parry Sound District Health Unit','The District of Algoma Health Unit','Sudbury and District Health Unit','Porcupine Health Unit'],
    "North West":['Northwestern Health Unit','Thunder Bay']  
}

In [ ]:
Regions = {
    "West":["Erie St. Clair","South West","Waterloo Wellington","Hamilton Niagara Haldimand Brant"],
    "Central":["Central West","Mississauga Halton","Central","North Simcoe Muskoka"],
    "Toronto":["Toronto Central"],
    "East": ["Central East","South East","Champlain"],
    "North": ["North East","North West"]
}

In [ ]:
url="https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1476137037"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
for item in PHU:
    df.loc[df.region.isin(PHU[item]),'LHIN'] = item

for item in Regions:
    df.loc[df.LHIN.isin(Regions[item]),'Region'] = item


    
df = df.drop(df.loc[df.region == 'Ontario'].index)

regions = df.region.values

df.region = [thing.replace("Health Unit","") for thing in regions]

fig = go.Figure()

df_t = df.groupby('region')['value'].sum().sort_values()

fig.add_trace(go.Bar(y=df_t.index, x=df_t.values, orientation='h', text=df_t.values, textposition='inside',marker_color="#413C90"))

df_t = df.groupby('LHIN')['value'].sum().sort_values()

fig.add_trace(go.Bar(y=df_t.index, x=df_t.values, orientation='h',text=df_t.values, textposition='inside',visible=False,marker_color="#413C90"))

df_t = df.groupby('Region')['value'].sum().sort_values()

fig.add_trace(go.Bar(y=df_t.index, x=df_t.values, orientation='h',text=df_t.values, textposition='inside',visible=False, marker_color="#413C90"))

fig.update_layout(
    xaxis =  {'showgrid': False},
    yaxis = {'showgrid': False},      
    title={'text':f"Total Cases by Public Health Unit",
            'y':0.98,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    ),
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0,
        y=-0.1,
        buttons=list([
            dict(label="PHU",
                 method="update",
                 args=[{"visible": [True, False, False]},
                       {"title": "Total Cases by Public Health Unit"}]),
            dict(label="LHIN",
                 method="update",
                 args=[{"visible": [False, True, False]},
                       {"title": "Total Cases by LHIN"}]),
            dict(label="Region",
                 method="update",
                 args=[{"visible": [False, False, True]},
                       {"title": "Total Cases by LHIN Region"}]),
        ]),
    )
])

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=2137633660"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df = df.loc[df.region=='Toronto']
df = df.sort_values('date')
df.tail()

In [ ]:

fig = go.Figure()

ttype = df.transportation_type.unique()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['value'].tail(3).values.mean(),
    number = {'suffix': "% of baseline", "font": {"size": 70}},
),
             )

fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Toronto Mobility<br><span style='font-size:0.5em;color:gray'>Last Updated: {df.date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 100,
                  }}, 
        ]
                         }})

for thing in ttype:
    temp = df.loc[df.transportation_type == thing]
    fig.add_trace(go.Scatter(name=thing,x=temp.date,y=temp['value'],opacity=0.5))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)


fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
)

In [ ]:
url="https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=812152023"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df = df.loc[df.region == 'Ontario']
df = df.sort_values(['region', 'category', 'date'])
df.head()

In [ ]:
df.category.unique()

In [ ]:
fig = go.Figure()

ttype = df.category.unique()

fig.add_trace(go.Scatter(name=ttype[0],x=df.loc[df.category == ttype[0]].date,y=df.loc[df.category == ttype[0]]['value']))

for item in ttype[1:]:
    fig.add_trace(go.Scatter(name=item,x=df.loc[df.category == item].date,y=df.loc[df.category == item]['value'],visible=False))


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False},      
    title={'text':f"{ttype[0]} {int(df.loc[df.category == ttype[0]]['value'].tail(1))}%",
            'y':0.99,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    ),
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="Grocery",
                 method="update",
                 args=[{"visible": [True, False, False, False, False, False]},
                       {"title": f"{ttype[0]} {int(df.loc[df.category == ttype[0]]['value'].tail(1))}%"}]),
            dict(label="Parks",
                 method="update",
                 args=[{"visible": [False, True, False, False, False, False]},
                       {"title": f"{ttype[1]} {int(df.loc[df.category == ttype[1]]['value'].tail(1))}%"}]),
            dict(label="Residential",
                 method="update",
                 args=[{"visible": [False, False, True, False, False, False]},
                       {"title": f"{ttype[2]} {int(df.loc[df.category == ttype[2]]['value'].tail(1))}%"}]),
            dict(label="Retail",
                 method="update",
                 args=[{"visible": [False, False, False, True, False, False]},
                       {"title": f"{ttype[3]} {int(df.loc[df.category == ttype[3]]['value'].tail(1))}%"}]),
            
            dict(label="Transit",
                 method="update",
                 args=[{"visible": [False, False, False, False, True, False]},
                       {"title": f"{ttype[4]} {int(df.loc[df.category == ttype[4]]['value'].tail(1))}%"}]),
            
            dict(label="Work",
                 method="update",
                 args=[{"visible": [False, False, False, False, False, True]},
                       {"title": f"{ttype[5]} {int(df.loc[df.category == ttype[5]]['value'].tail(1))}%"}]),
        ]),
    )
])

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1694584755"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['date','lhin'])
df.head()

In [ ]:
last_date = df.tail(1).date.values[0]
df = df.loc[df.date == last_date]


fig = go.Figure()

fig.add_trace(
    go.Table(
        header=dict(
            values=["LHIN", "Residual Beds", "Residual Ventilators"],
            font=dict(size=10),
            align="left",
            fill_color='#E0DFED'
        ),
        cells=dict(
            values=[df[k].tolist() for k in df[['lhin', 'residual_beds', 'residual_ventilators']].columns[:]],
            align = "left",
        fill_color='#E0DFED')
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
)

In [ ]:
df.head()

In [ ]:
last_date = df.tail(1).date.values[0]
df = df.loc[df.date == last_date]

df.rename(columns={"confirmed_positive":"Confirmed Covid"},inplace=True)
df.rename(columns={"suspected_covid":"Suspected Covid"},inplace=True)
df.rename(columns={"non_covid":"Not Covid"},inplace=True)

fig = go.Figure()

for thing in ['Confirmed Covid','Suspected Covid','Not Covid']:
    fig.add_trace(go.Bar(name=thing,y=df.lhin, x=df[thing],orientation='h'))

fig.update_layout(barmode='stack')

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h"
)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1313299474"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['date'])
df.head()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['residual_beds'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"ICU Beds Left<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['residual_beds'].iloc[-2],
                  }}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.date,y=df['residual_beds'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Residual Beds Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['residual_ventilators'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Ventilators Left<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['residual_ventilators'].iloc[-2],
                  }}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.date,y=df['residual_ventilators'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=22,
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Residual Ventilators Over Time"},]),
        ]),
    )])

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1PTVSFZwSVWldmDpFEO_xXxTB-xSFtGq3pDPUKeLDEec/export?format=csv&id=1PTVSFZwSVWldmDpFEO_xXxTB-xSFtGq3pDPUKeLDEec&gid=0"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

In [ ]:
df.SES.replace(1,"1 (least deprived)",inplace=True)
df.SES.replace(5,"5 (most deprived)",inplace=True)


fig = go.Figure()

types = df['case status'].unique()

socio = ["1 (least deprived)",2,3,4,"5 (most deprived)"]

colors = ['blue', 'red']

colors = ['#B9DDF1','#8BBADC','#6798C1','#4776A4','#2A5783']

for ttype, color in zip(socio,colors):
    temp = df.loc[df.SES == ttype]
    fig.add_trace(go.Bar(name=ttype,x=types,y=temp['value'].values,text=temp['value'].values,textposition='auto',marker_color=color))


fig.update_layout(barmode='stack')

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h"
)

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': True,'visible':True}, 
    title={'text':f"Distribution of COVID19 Cases by Neighbourhood Socioeconmic Status",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=2041757828"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

In [ ]:
Canada = ['Ontario', 'BC', 'Alberta', 'Quebec', 'Saskatchewan',
       'Nova Scotia', 'NL', 'New Brunswick', 'Manitoba', 'Canada']
International = ['Ontario','Canada',
       'Italy', 'Korea, South', 'Spain', 'United Kingdom', 'France', 'US']

fig = go.Figure()

for item in Canada:
    temp = df.loc[df.region == item]
    fig.add_trace(go.Scatter(name=item,x=temp.date_shifted,y=temp.average))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': True,'visible':True, 'type': 'log'}, 
    title={'text':f"Provincial Comparison: COVID19 Reported Cases Per Day",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
    legend_orientation="h"
)

In [ ]:
fig = go.Figure()

for item in International:
    temp = df.loc[df.region == item]
    fig.add_trace(go.Scatter(name=item,x=temp.date_shifted,y=temp.average))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': True,'visible':True, 'type': 'log'}, 
    title={'text':f"International Comparison: COVID19 Reported Cases Per Day",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h"
)

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=169560279"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

In [ ]:
Canada = ['Ontario', 'BC', 'Alberta', 'Quebec', 'Saskatchewan',
       'Nova Scotia', 'NL', 'New Brunswick', 'Manitoba', 'Canada']
International = ['Ontario','Canada',
       'Italy', 'Korea, South', 'Spain', 'United Kingdom', 'France', 'US']

fig = go.Figure()

for item in Canada:
    temp = df.loc[df.region == item]
    fig.add_trace(go.Scatter(name=item,x=temp.date_shifted,y=temp.average))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': True,'visible':True, 'type': 'log'}, 
    title={'text':f"Provincial Comparison: COVID19 Reported Deaths Per Day",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
    legend_orientation="h"
)

In [ ]:
fig = go.Figure()

for item in International:
    temp = df.loc[df.region == item]
    fig.add_trace(go.Scatter(name=item,x=temp.date_shifted,y=temp.average))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': True,'visible':True, 'type': 'log'}, 
    title={'text':f"International Comparison: COVID19 Reported Deaths Per Day",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h"
)

In [ ]:
url = " https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1167743117"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

In [ ]:
df = df.loc[df.region == 'Ontario']

fig = go.Figure()

fig.add_trace(go.Bar(x=df.date,y=df.daily_deaths,text=df.daily_deaths,textposition='auto', marker_color='#4F4B99'))

fig.update_layout(
    title={'text':"Daily Deaths in Ontario: Comparison of COVID-19 and Estimates of non-COVID-19 Leading Causes",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=40, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    showlegend=False
)

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=df.date.head(1).values[0],
            y0=8,
            x1=df.date.tail(1).values[0],
            y1=8,
            line=dict(
                color="grey",
                width=1,
            ),
    )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=df.date.head(1).values[0],
            y0=14,
            x1=df.date.tail(1).values[0],
            y1=14,
            line=dict(
                color="grey",
                width=1,
            ),
    )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=df.date.head(1).values[0],
            y0=16,
            x1=df.date.tail(1).values[0],
            y1=16,
            line=dict(
                color="grey",
                width=1,
            ),
    )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=df.date.head(1).values[0],
            y0=56,
            x1=df.date.tail(1).values[0],
            y1=56,
            line=dict(
                color="grey",
                width=1,
            ),
    )

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=df.date.head(1).values[0],
            y0=83,
            x1=df.date.tail(1).values[0],
            y1=83,
            line=dict(
                color="grey",
                width=1,
            ),
    )

fig.update_shapes(dict(xref='x', yref='y'))

fig.add_trace(go.Scatter(
    x=[df.date.head(3).values[1], df.date.head(3).values[1], df.date.head(3).values[1],df.date.head(3).values[1],df.date.head(3).values[1],df.date.head(3).values[1]],
    y=[8+1,14+1,16+1,56+1,83+1],
    text=["Diabetes","Cerebrovascular Disease","Accidents / Injuries","Heart Disease","All Cancers"],
    mode="text",
))

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U/export?format=csv&id=1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U&gid=0"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['LTC Deaths'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"LTC Deaths<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['LTC Deaths'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['LTC Deaths'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "LTC Deaths Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['LTC Cases Total'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"LTC Cases<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['LTC Cases Total'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['LTC Cases Total'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "LTC Cases Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
),
             )


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#FFF"
    )
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor='#FFF',
    paper_bgcolor="#FFF",
)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Staff'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"LTC Staff Infected<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Staff'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['Staff'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "LTC Staff Infections Over Time"},]),
        ]),
    )])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['Hospital Staff'].tail(1).values[0],
),
             )





fig.update_layout(
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Hospital Staff Infected<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['Hospital Staff'].iloc[-2],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})



fig.add_trace(go.Scatter(x=df.Date,y=df['Hospital Staff'],marker_color='#497787', visible=False))

fig.update_layout(
    xaxis =  {'showgrid': False,'visible':False},
    yaxis = {'showgrid': False,'visible':False}, 
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#DFE7EA',
    paper_bgcolor="#DFE7EA",
updatemenus=[
    dict(
        type="buttons",
        direction="right",
        active=0,
        x=0.57,
        y=-0.1,
        buttons=list([
            dict(label="KPI",
                 method="update",
                 args=[{"visible": [True, False]},
                       {"title": ""}
                      ]),
            dict(label="Trend",
                 method="update",
                 args=[{"visible": [False, True]},
                       {"title": "Hospital Staff Infections Over Time"},]),
        ]),
    )])

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

from scipy import stats as sps
from scipy.interpolate import interp1d

from IPython.display import clear_output

%config InlineBackend.figure_format = 'retina'

In [ ]:
url = "https://raw.githubusercontent.com/ishaberry/Covid19Canada/master/cases.csv"
s=requests.get(url).content
cases_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
cases_df['date'] = pd.to_datetime(cases_df['date_report'])
cases_df.head()

In [ ]:
province_df = cases_df.groupby(['province', 'date'])['case_id'].count()
province_df.index.rename(['health_region', 'date'], inplace=True)
hr_df = cases_df.groupby(['health_region', 'date'])['case_id'].count()
canada_df = pd.concat((province_df, hr_df))
canada_df.head()

In [ ]:
prov_name = 'Ontario'

def prepare_cases(cases):
    new_cases = cases

    smoothed = new_cases.rolling(7,
        win_type='gaussian',
        min_periods=1,
        center=False).mean(std=2).round()
    
    zeros = smoothed.index[smoothed.eq(0)]
    if len(zeros) == 0:
        idx_start = 0
    else:
        last_zero = zeros.max()
        idx_start = smoothed.index.get_loc(last_zero) + 1
    smoothed = smoothed.iloc[idx_start:]
    original = new_cases.loc[smoothed.index]
    
    return original, smoothed

cases = canada_df.xs(prov_name).rename(f"{prov_name} cases")

original, smoothed = prepare_cases(cases)

original.plot(title=f"{prov_name} New Cases per Day",
               c='k',
               linestyle=':',
               alpha=.5,
               label='Actual',
               legend=True,
             figsize=(600/72, 400/72))

ax = smoothed.plot(label='Smoothed',
                   legend=True)
ax.get_figure().set_facecolor('w')

In [ ]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=1086007065"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
df = df.loc[df.region == 'Ontario']

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['ML'].tail(1).values[0],
    number = {'font': {'size': 60}},))

fig.add_trace(go.Scatter(x=df.date,y=df.ML,marker_color='#5E5AA1',visible=True,opacity=0.5))

fig.add_trace(go.Scatter(x=df.date,y=df.Low,
    fill=None,
    mode='lines',
    line_color='#4F4B99',opacity=0.1
    ))
fig.add_trace(go.Scatter(x=df.date,y=df.High,
    fill='tonexty', 
    mode='lines', line_color='#4F4B99',opacity=0.1))


fig.update_layout(
    showlegend=False,
    template = {'data' : {'indicator': [{
        'title' : {"text": f"<span style='font-size:0.8em>R<sub>t</sub> value</span><br><span style='font-size:0.8em;color:gray'>Last Updated: {df.date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['ML'].tail(2).values[0],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.add_shape(
            type="line",
            xref="paper",
            yref="y",
            x0=0,
            y0=1,
            x1=1,
            y1=1,
            line=dict(
                color="red",
                width=2,
            ),
        )

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",
    )



In [ ]:
url = "https://docs.google.com/spreadsheets/d/1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U/export?format=csv&id=1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U&gid=689073638"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
df.groupby('Date')['LTC Home'].count()

In [ ]:
df = df.loc[df.PHU == 'York Region Public Health']

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = df.groupby('Date')['LTC Home'].count().tail(1).values[0],
    number = {'font': {'size': 60}},))



fig.update_layout(
    showlegend=False,
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Number of LTC Home Outbreak<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number",
    }, 
        ]
                         }})


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",
    )



In [ ]:
df = df.loc[df.PHU == 'York Region Public Health']

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = df.groupby('Date')['Confirmed Resident Cases'].sum().tail(1).values[0],
    number = {'font': {'size': 60}},))



fig.update_layout(
    showlegend=False,
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Confirmed Resident Cases<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number",
    }, 
        ]
                         }})


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E4F7FD',
    paper_bgcolor="#E4F7FD",
    legend_orientation="h",
    )



In [ ]:
df = df.loc[df.PHU == 'York Region Public Health']

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = df.groupby('Date')['Confirmed Staff Cases'].sum().tail(1).values[0],
    number = {'font': {'size': 60}},))



fig.update_layout(
    showlegend=False,
    template = {'data' : {'indicator': [{
        'title' : {"text": f"Confirmed Staff Cases<br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number",
    }, 
        ]
                         }})


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",
    )



In [ ]:
url = "https://docs.google.com/spreadsheets/d/1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U/export?format=csv&id=1pWmFfseTzrTX06Ay2zCnfdCG0VEJrMVWh-tAU9anZ9U&gid=0"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'New Cases']]
df = df.dropna()
df.head()

In [ ]:
import numpy as np
from scipy import stats as sps
from scipy.interpolate import interp1d

In [ ]:
def prepare_cases(cases):
    # modification - Isha Berry et al.'s data already come in daily
    #new_cases = cases.diff()
    new_cases = cases

    smoothed = new_cases.rolling(7,
        win_type='gaussian',
        min_periods=1,
        # Alf: switching to right-aligned instead of centred to prevent leakage of
        # information from the future
        #center=True).mean(std=2).round()
        center=False).mean(std=2).round()

    zeros = smoothed.index[smoothed.eq(0)]
    if len(zeros) == 0:
        idx_start = 0
    else:
        last_zero = zeros.max()
        idx_start = smoothed.index.get_loc(last_zero) + 1
    smoothed = smoothed.iloc[idx_start:]
    original = new_cases.loc[smoothed.index]
    return original, smoothed

# We create an array for every possible value of Rt
R_T_MAX = 12
r_t_range = np.linspace(0, R_T_MAX, R_T_MAX*100+1)

# Gamma is 1/serial interval
# https://wwwnc.cdc.gov/eid/article/26/6/20-0357_article
GAMMA = 1/4

def get_posteriors(sr, window=7, min_periods=1):
    lam = sr[:-1].values * np.exp(GAMMA * (r_t_range[:, None] - 1))

    # Note: if you want to have a Uniform prior you can use the following line instead.
    # I chose the gamma distribution because of our prior knowledge of the likely value
    # of R_t.

    # prior0 = np.full(len(r_t_range), np.log(1/len(r_t_range)))
    prior0 = np.log(sps.gamma(a=3).pdf(r_t_range) + 1e-14)

    likelihoods = pd.DataFrame(
        # Short-hand way of concatenating the prior and likelihoods
        data = np.c_[prior0, sps.poisson.logpmf(sr[1:].values, lam)],
        index = r_t_range,
        columns = sr.index)

    # Perform a rolling sum of log likelihoods. This is the equivalent
    # of multiplying the original distributions. Exponentiate to move
    # out of log.
    posteriors = likelihoods.rolling(window,
                                     axis=1,
                                     min_periods=min_periods).sum()
    posteriors = np.exp(posteriors)

    # Normalize to 1.0
    posteriors = posteriors.div(posteriors.sum(axis=0), axis=1)

    return posteriors

def highest_density_interval(pmf, p=.95):
    # If we pass a DataFrame, just call this recursively on the columns
    if(isinstance(pmf, pd.DataFrame)):
        return pd.DataFrame([highest_density_interval(pmf[col]) for col in pmf],
                            index=pmf.columns)

    cumsum = np.cumsum(pmf.values)
    best = None
    for i, value in enumerate(cumsum):
        for j, high_value in enumerate(cumsum[i+1:]):
            if (high_value-value > p) and (not best or j<best[1]-best[0]):
                best = (i, i+j+1)
                break

    low = pmf.index[best[0]]
    high = pmf.index[best[1]]
    return pd.Series([low, high], index=['Low', 'High'])

results = None

new, smoothed = prepare_cases(df.set_index('Date')['New Cases'])
try:
    posteriors = get_posteriors(smoothed)
except Exception as e:
    print(e)
hdis = highest_density_interval(posteriors)
most_likely = posteriors.idxmax().rename('ML')
result = pd.concat([most_likely, hdis], axis=1).reset_index()
if results is None:
    results = result
else:
    results = results.append(result)


In [ ]:
df = results

In [ ]:
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = df['ML'].tail(1).values[0],
    number = {'font': {'size': 60}},))

fig.add_trace(go.Scatter(x=df.Date,y=df.ML,marker_color='#5E5AA1',visible=True,opacity=0.5))

fig.add_trace(go.Scatter(x=df.Date,y=df.Low,
    fill=None,
    mode='lines',
    line_color='#4F4B99',opacity=0.1
    ))
fig.add_trace(go.Scatter(x=df.Date,y=df.High,
    fill='tonexty', 
    mode='lines', line_color='#4F4B99',opacity=0.1))


fig.update_layout(
    showlegend=False,
    template = {'data' : {'indicator': [{
        'title' : {"text": f"<span style='font-size:0.8em>R<sub>t</sub> value</span><br><span style='font-size:0.8em;color:gray'>Last Updated: {df.Date.tail(1).values[0].astype('M8[D]')}</span><br>"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': df['ML'].tail(2).values[0],
                  'increasing': {'color':'red'},
                  'decreasing': {'color':'green'}}}, 
        ]
                         }})


fig.update_layout(
    xaxis =  {'showgrid': False,'visible':True},
    yaxis = {'showgrid': False,'visible':True},      
    title={'text':f"",
            'y':0.95,
            'x':0.5,
           'xanchor': 'center',
            'yanchor': 'top'},
    font=dict(
        family="Roboto",
        color="#000"
    )
)

fig.add_shape(
            type="line",
            xref="paper",
            yref="y",
            x0=0,
            y0=1,
            x1=1,
            y1=1,
            line=dict(
                color="red",
                width=2,
            ),
        )

fig.update_layout(
    margin=dict(l=0, r=10, t=30, b=50),
    plot_bgcolor='#E0DFED',
    paper_bgcolor="#E0DFED",
    legend_orientation="h",
    )



In [7]:
import json
import plotly.express as px

In [8]:
url = "https://docs.google.com/spreadsheets/u/0/d/1TH8vQCljcNGWzq4MXRNXGp243izKGpWq5adeEbjNBlU/export?format=csv&id=1TH8vQCljcNGWzq4MXRNXGp243izKGpWq5adeEbjNBlU&gid=0"
s=requests.get(url).content
loc = pd.read_csv(io.StringIO(s.decode('utf-8')))
loc.head()

,PHU Locations,Latitude,Longitude
0,York Regional Health Unit,44.00481,-79.4973
1,Windsor-Essex County Health Unit,42.12471,-82.7734
2,Wellington-Dufferin-Guelph Health Unit,43.85650,-80.3925
3,Waterloo Health Unit,43.45118,-80.5101
4,Timiskaming Health Unit,47.63172,-79.9219


In [25]:
url = "https://docs.google.com/spreadsheets/d/19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA/export?format=csv&id=19LFZWy85MVueUm2jYmXXE6EC3dRpCPGZ05Bqfv5KyGA&gid=166537997"
s=requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df = df.merge(loc, left_on='region', right_on='PHU Locations')
df.head()

,region,cases,deaths,outbreaks,critical_care_beds,critical_care_patients,vented_beds,vented_patients,suspected_covid,suspected_covid_ventilator,confirmed_positive,confirmed_negative,confirmed_positive_ventilator,non_covid,residual_beds,residual_ventilators,PHU Locations,Latitude,Longitude
0,The District of Algoma Health Unit,13,0,0,102,59,128,18,15,0,2,0,1,42,43,110,The District of Algoma Health Unit,46.35371,-83.1094
1,Brant County Health Unit,96,3,1,271,186,349,62,29,0,8,0,7,149,85,287,Brant County Health Unit,43.07961,-80.3479
2,Durham Regional Health Unit,995,113,7,163,126,230,69,41,0,22,0,20,63,37,161,Durham Regional Health Unit,44.00116,-78.8934
3,Grey Bruce Health Unit,82,0,2,187,118,209,34,23,0,9,0,8,86,69,175,Grey Bruce Health Unit,44.32925,-81.0005
4,Haldimand-Norfolk Health Unit,194,30,1,458,304,558,96,52,0,17,0,15,235,154,462,Haldimand-Norfolk Health Unit,42.72988,-80.3315


In [10]:
phu = json.load(open("HR_035a18m_e.geojson"))

In [34]:
df['text'] = "<b>" + (df['region']).astype(str) + "</b>" + '<br>Confirmed: ' + (df['cases']).astype(str) + '<br>Deaths: ' + (df['deaths']).astype(str) + '<br># of LTC Outbreaks: ' + (df['outbreaks']).astype(str)


fig = go.Figure()

token ="pk.eyJ1IjoiZmFyYm9kYWIiLCJhIjoiY2s4OTRmejFjMDFsZzNmbXpzamFuOXZnMiJ9.Um1dSPlv154QcsK45jH-Dw"

fig.add_trace(go.Scattermapbox(
        lat=df.Latitude,
        lon=df.Longitude,
        mode='markers',
        visible=True,
        name="Cases",
        text=df.text,
        hoverinfo='text',
        marker=go.scattermapbox.Marker(
            size=df.cases/10,
            sizemode = 'area',
            sizemin=4,
            color='red',
            opacity=0.5
        ),
    ))

fig.add_trace(go.Scattermapbox(
        lat=df.Latitude,
        lon=df.Longitude,
        mode='markers',
        text=df.text,
        visible=False,
        name="Deaths",
        hoverinfo='text',
        marker=go.scattermapbox.Marker(
            size=df.deaths,
            sizemode = 'area',
            sizemin=4,
            color='yellow',
            opacity=0.5
        ),
    ))

fig.add_trace(go.Scattermapbox(
        lat=df.Latitude,
        lon=df.Longitude,
        mode='markers',
        visible=False,
        name="Outbreaks",
        text=df.text,
        hoverinfo='text',
        marker=go.scattermapbox.Marker(
            size=df.outbreaks*5,
            sizemode ='area',
            sizemin=4,
            color='green',
            opacity=0.5
        ),
    ))



fig.update_layout(mapbox_style="dark")
fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=token,
        bearing=0,
        center=dict(
            lat=47.2,
            lon=-83.3
        ),
        pitch=0,
        zoom=4
    ),
)

fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction="right",
            pad={"r": 10, "t": 10},
            showactive=True,
            y=0.01,
            x=0.5,
            xanchor='center',
            yanchor='top',
            buttons=list([
                dict(label="Cases",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Cases"}]),
                dict(label="Deaths",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {"title": "Deaths"}]),
                dict(label="Outbreaks",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {"title": "Outbreaks"}]),
            ]),
        )
    ])


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="cases",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()
